In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { max-width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.output_area { max-width:100% !important; }</style>"))
display(HTML("<style>.input_area { max-width:100% !important; }</style>"))

# Case study: Analiza danych tekstowych (_text mining_)

Pobierz z _Wikipedii_ treść 11 artykułów.

In [4]:
#!pip install requests

# from urllib.request import urlopen
import requests

wiki_url = "http://en.wikipedia.org/wiki/"
titles = [
    "Integral", 
    "Riemann_integral", 
    "Riemann-Stieltjes_integral", 
    "Derivative",
    "Limit_of_a_sequence", 
    "Edvard_Munch", 
    "Vincent_van_Gogh", 
    "Jan_Matejko",
    "Lev_Tolstoj", 
    "Franz_Kafka", 
    "J._R._R._Tolkien"
]

urls = [wiki_url + title for title in titles]

# articles = [urlopen(url).read() for url in urls]
articles = [requests.get(url).content for url in urls ]

# Wyświetlamy pierwsze 200 znaków pierwszego artykułu
print(articles[0][:200])

b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-'


Wczytaliśmy cały kod strony artykułu (włącznie z informacjami o nagłówkach, paskami nawigacyjnymi, ... ). Nas interesuje wyłącznie sama treść artykułów. W jaki sposób można wyciągnąć z kodu źródłowego strony wybrane informacje?


## Scrapping

Proces mający na celu wyciągnięcie z nieustrukturyzowanego zbioru danych tekstowych (a kod źródłowy strony internetowej jest takim zbiorem) wybranych informacji nazywany jest **scrapingiem**. W Pythonie najpopularniejszą biblioteką do scrapingu jest `BeautifulSoup`.

Żeby "zeskrapować" jakiś tekst, najpierw trzeba rozpoznać w nim charakterystyczne elementy. Przeglądając kod źródłowy artykułów zauważymy, że artykuły na wiki są zamknięte w elemencie `<div>` o `id=bodyContent`, a poszczególne akapity artykułu to po prostu paragrafy (`<p>`). 

Czyli teraz z całej tej pobranej treści chcemy wyciągnąć wyłącznie elementy `<p>` znajdujące się wewnątrz elementu `<div>` o id `"bodyContent"`. Do tego właśnie służy biblioteka `BeautifulSoup`.

In [5]:
from bs4 import BeautifulSoup

articles_paragraphs = [BeautifulSoup(article).find("div", id="bodyContent").find_all("p") for article in articles]

# wyświetlmy pierwszy paragraf pierwszego artykułu
print(articles_paragraphs[0][0])

<p>In <a href="/wiki/Mathematics" title="Mathematics">mathematics</a>, an <b>integral</b> is the continuous analog of a <a href="/wiki/Summation" title="Summation">sum</a>, which is used to calculate <a href="/wiki/Area" title="Area">areas</a>, <a href="/wiki/Volume" title="Volume">volumes</a>, and their generalizations. Integration, the process of computing an integral, is one of the two fundamental operations of <a href="/wiki/Calculus" title="Calculus">calculus</a>,<sup class="reference" id="cite_ref-1"><a href="#cite_note-1">[a]</a></sup> the other being <a href="/wiki/Derivative" title="Derivative">differentiation</a>. Integration started as a method to solve problems in mathematics and <a href="/wiki/Physics" title="Physics">physics</a>, such as finding the area under a curve, or determining displacement from velocity. Today integration is used in a wide variety of scientific fields.
</p>


Każdy paragraf jest reprezentowany jako obiekty klasy **Tag** biblioteki `BeautifulSoup`.

In [6]:
print(type(articles_paragraphs[0][0]))

<class 'bs4.element.Tag'>


Zrzutujmy paragrafy na typ string.

In [7]:
articles_paragraphs = [[str(paragraph) for paragraph in paragraphs] for paragraphs in articles_paragraphs]

Teraz każdy artykuł mamy porozbijany na zbiór paragrafów. Sklejmy te paragrafy tak, żeby artykuły znów stały się jednym ciągiem znaków.

In [8]:
scraped_articles = ["".join(paragraphs) for paragraphs in articles_paragraphs]

# wyświetlmy pierwsze 200 znaków pierwszego, tak przetworzonego artykułu
print(scraped_articles[0][:200])

<p>In <a href="/wiki/Mathematics" title="Mathematics">mathematics</a>, an <b>integral</b> is the continuous analog of a <a href="/wiki/Summation" title="Summation">sum</a>, which is used to calculate 


Pozbądźmy się z tekstu znaczników html, tak żeby w paragrafach został już czysty tekst. Do tego celu użyjemy biblioteki `re` (wyrażenia regularne). Za wzorzec znacznika html przyjmujemy `<.+?>` (znak "<" po którym następuje jedne lub więcej znaków, kończący się znakiem ">" - leniwe)

In [9]:
import re

cleaned_articles = [re.sub("<.+?>", "", article) for article in scraped_articles]

# wyświetlmy piersze 5000 znaków pierwszego, oczyszczonego w ten sposób artykuł
print(cleaned_articles[0][:5000])

In mathematics, an integral is the continuous analog of a sum, which is used to calculate areas, volumes, and their generalizations. Integration, the process of computing an integral, is one of the two fundamental operations of calculus,[a] the other being differentiation. Integration started as a method to solve problems in mathematics and physics, such as finding the area under a curve, or determining displacement from velocity. Today integration is used in a wide variety of scientific fields.
The integrals enumerated here are called definite integrals, which can be interpreted as the signed area of the region in the plane that is bounded by the graph of a given function between two points in the real line. Conventionally, areas above the horizontal axis of the plane are positive while areas below are negative. Integrals also refer to the concept of an antiderivative, a function whose derivative is the given function; in this case, they are also called indefinite integrals. The funda

## Preprocessing

### Zamiana wielkich liter na małe

In [10]:
cleaned_articles = [a.lower() for a in cleaned_articles]

# wyświetlmy piersze 200 znaków pierwszego artykułu
print(cleaned_articles[0][:200])

in mathematics, an integral is the continuous analog of a sum, which is used to calculate areas, volumes, and their generalizations. integration, the process of computing an integral, is one of the tw


### Tokenizacja

In [11]:
from nltk.tokenize import word_tokenize

cleaned_articles = [word_tokenize(article) for article in cleaned_articles]

# wyświetlmy pierwsz 20 tokenów pierwszego artykułu
print(cleaned_articles[0][:20])

['in', 'mathematics', ',', 'an', 'integral', 'is', 'the', 'continuous', 'analog', 'of', 'a', 'sum', ',', 'which', 'is', 'used', 'to', 'calculate', 'areas', ',']


### Usuwanie znaków interpunkcyjnych

In [12]:
import string

cleaned_articles = [[token for token in article if token not in string.punctuation] for article in cleaned_articles]

# wyświetlmy piersze 20 tokenów pierwszego artykułu
print(cleaned_articles[0][:20])

['in', 'mathematics', 'an', 'integral', 'is', 'the', 'continuous', 'analog', 'of', 'a', 'sum', 'which', 'is', 'used', 'to', 'calculate', 'areas', 'volumes', 'and', 'their']


### Usuwanie __stopwords__

In [13]:
from nltk.corpus import stopwords

stopwords_list = stopwords.words('english')

cleaned_articles = [[token for token in article if token not in stopwords_list] for article in cleaned_articles]

# wyświetlmy pierwsze 20 tokenów pierwszego artykułu
print(cleaned_articles[0][:20])

['mathematics', 'integral', 'continuous', 'analog', 'sum', 'used', 'calculate', 'areas', 'volumes', 'generalizations', 'integration', 'process', 'computing', 'integral', 'one', 'two', 'fundamental', 'operations', 'calculus', 'differentiation']


### Stemming

In [14]:
from nltk.stem import PorterStemmer  # najpopularniejszy stemmer

stemmer = PorterStemmer()
cleaned_articles = [[stemmer.stem(token) for token in article] for article in cleaned_articles]

# wyświetlmy pierwsze 20 tokenów pierwszego artykułu
print(cleaned_articles[0][:20])

['mathemat', 'integr', 'continu', 'analog', 'sum', 'use', 'calcul', 'area', 'volum', 'gener', 'integr', 'process', 'comput', 'integr', 'one', 'two', 'fundament', 'oper', 'calculu', 'differenti']


## TF-IDF Embedding (osadzanie/wektoryzacja)

In [15]:
#!pip install gensim

from gensim.corpora.dictionary import Dictionary
from gensim.models.tfidfmodel import TfidfModel

dictionary = Dictionary(cleaned_articles)
bow_corpus = [dictionary.doc2bow(article) for article in cleaned_articles]
tfidf_model = TfidfModel(bow_corpus)

In [16]:
# stworzenie całego korpusu w modelu TF-IDF
tfidf_corpus = tfidf_model[bow_corpus]

# Wyświetlmy pierwsze 100 tokenów pierwszego artykułu
print(tfidf_corpus[0][:100])

[(1, 0.009205551443916363), (3, 0.009205551443916363), (4, 0.009205551443916363), (5, 0.009205551443916363), (6, 0.009205551443916363), (7, 0.001097691095942296), (8, 0.009205551443916363), (9, 0.027242083451491473), (10, 0.009205551443916363), (12, 0.006544550314620297), (13, 0.009205551443916363), (15, 0.009205551443916363), (16, 0.0003658970319807654), (17, 0.0007703774352945387), (18, 0.0007703774352945387), (19, 0.001222548056028167), (20, 0.001222548056028167), (21, 0.0017351768259862108), (22, 0.009205551443916363), (23, 0.006544550314620297), (24, 0.009205551443916363), (25, 0.0034703536519724216), (26, 0.02617820125848119), (27, 0.0017351768259862108), (28, 0.009205551443916363), (29, 0.009205551443916363), (30, 0.002326963310309385), (31, 0.009205551443916363), (33, 0.018411102887832726), (34, 0.009205551443916363), (35, 0.001222548056028167), (36, 0.0030268981612768305), (37, 0.0017351768259862108), (38, 0.0017351768259862108), (39, 0.0017351768259862108), (40, 0.00173517682

Wyświetlmy pierwsze 10 najczęściej występujących tokenów z pierwszego artykułu

In [17]:
first_article = sorted(tfidf_model[bow_corpus[0]], key=lambda x: x[1], reverse=True)[:10]
first_article

[(501, 0.7294824568677163),
 (417, 0.2652738173752699),
 (375, 0.1786432625249147),
 (737, 0.14465096874855807),
 (509, 0.13980777067167235),
 (292, 0.1396630043089526),
 (141, 0.1288777202148291),
 (161, 0.12712972277362689),
 (237, 0.12469911099013627),
 (831, 0.10474725323171445)]

Co to za słowa ?

In [18]:
[dictionary[token[0]] for token in first_article]

['integr',
 'function',
 'f',
 'riemann',
 'interv',
 'differenti',
 'antideriv',
 'b',
 'comput',
 'surfac']

## Analiza semantyczna

Jednym z kolejnych etapów może być _analiza semantyczna_ (czyli znaczeniowa) przetwarzanych treści. Częstym zadaniem, które w tym miejscu pojawiaja się jest dopasowywanie tytułów (kategorii) do treści. Jednym z algorytmów odnoszących świetne wyniki w tej dziedzinie jest algorytm **LDA** (*ang. Latent Dirichlet allocation*), który do znalezienia najlepiej pasujących tytułów wykorzystuje analizę **PCA** podanego mu rozkładu **TF-IDF** lub **BoW**. Biblioteka `Gensim` posiada implementację tego algorytmu. Wystarczy przekazać do niej **słownik** oraz wynik zwrócony przez model **TF-IDF** lub **BoW**.

In [19]:
from gensim.models.ldamodel import LdaModel

lda_model = LdaModel(
    corpus=tfidf_corpus,
    id2word=dictionary,
    num_topics=30,
    random_state=100,
    update_every=1,
    chunksize=100,
    passes=10,
    alpha="auto"
)

print(lda_model)

/home/mdabrowski-phd/.local/lib/python3.10/site-packages/gensim/models/ldamodel.py:850: RuntimeWarning: overflow encountered in exp2
  perwordbound, np.exp2(-perwordbound), len(chunk), corpus_words


LdaModel<num_terms=7218, num_topics=30, decay=0.5, chunksize=100>


W modelu **LDA** przyjęliśmy `num_topics=30` co oznacza, że model będzie szukał 30 najlepszych tytułów dla całego korpusu. Tytuł będzie zbudowanych z najczęściej występujących w tokenów. Biblioteka `pyldavis` do wizualizacji wyników zwróconych przez model umożliwia wyświetlenie wyników na wykresie.

In [22]:
#!pip install pyldavis

import pyLDAvis
import pyLDAvis.gensim

In [23]:
pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim.prepare(
    lda_model, 
    tfidf_corpus, 
    dictionary, 
    mds="mmds", 
    R=20  # liczba tokenów branych pod uwagę podczas analizy tytułu dla wybranego artykułu
)

vis

/home/mdabrowski-phd/.local/lib/python3.10/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.003195  0.001465       1        1  14.592732
19     0.001556 -0.004017       2        1  13.478287
25     0.003552  0.001087       3        1  10.918995
16     0.000731  0.001849       4        1   9.271769
5      0.001032 -0.000677       5        1   7.523401
11    -0.000540 -0.001615       6        1   6.817320
28    -0.000711  0.001257       7        1   6.555847
1      0.002114  0.000442       8        1   6.236180
3     -0.001614 -0.001364       9        1   4.456246
10    -0.000307 -0.000058      10        1   1.288120
0     -0.000132  0.000084      11        1   0.943055
22    -0.000131  0.000078      12        1   0.943055
21    -0.000132  0.000079      13        1   0.943055
24    -0.000130  0.000081      14        1   0.943055
23    -0.000131  0.000085      15        1   0.943055
26    -0.000127  0.000078      16        1   0.943055
27    -0.000129  0.000078      17        1   0.943055
20    -0.000133  0.000084      18        1   0.943055
15    -0.000132  0.000081      19        1   0.943055
18    -0.000131  0.000085      20        1   0.943055
17    -0.000131  0.000085      21        1   0.943055
14    -0.000132  0.000085      22        1   0.943055
13    -0.000133  0.000086      23        1   0.943055
12    -0.000128  0.000078      24        1   0.943055
9     -0.000128  0.000083      25        1   0.943055
8     -0.000132  0.000079      26        1   0.943055
7     -0.000130  0.000076      27        1   0.943055
4     -0.000132  0.000081      28        1   0.943055
2     -0.000132  0.000082      29        1   0.943055
29    -0.000132  0.000083      30        1   0.943055, topic_info=         Term      Freq     Total Category  logprob  loglift
501    integr  0.000000  0.000000  Default  20.0000  20.0000
5497  tolstoy  0.000000  0.000000  Default  19.0000  19.0000
6027    kafka  0.000000  0.000000  Default  18.0000  18.0000
7137  tolkien  0.000000  0.000000  Default  17.0000  17.0000
2564    munch  0.000000  0.000000  Default  16.0000  16.0000
...       ...       ...       ...      ...      ...      ...
15       1040  0.000152  0.015968  Topic30  -8.8843   0.0089
16         11  0.000152  0.015969  Topic30  -8.8843   0.0088
17         12  0.000152  0.015972  Topic30  -8.8843   0.0086
18         13  0.000152  0.015972  Topic30  -8.8843   0.0086
19         14  0.000152  0.015973  Topic30  -8.8843   0.0086

[968 rows x 6 columns], token_table=Empty DataFrame
Columns: [Topic, Freq, Term]
Index: [], R=20, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 20, 26, 17, 6, 12, 29, 2, 4, 11, 1, 23, 22, 25, 24, 27, 28, 21, 16, 19, 18, 15, 14, 13, 10, 9, 8, 5, 3, 30])

Każdy ze znajdujących się po lewej stronie okręgów reprezentuje jakiś potencjalny tytuł (zbiór tokenów, z których ten tytuł można zbudować). Chcieliśmy znaleźć 30 propozycji tytułów. Algorytmowi udało się znaleźć 8 (co ma sens, ponieważ wczytaliśmy z wiki tylko 11 artykułów). Niektóre z nich (np. 3 i 4) nakładają się. Im bardziej tytuły będą od siebie odseparowane, tym lepiej. Po kliknięciu w wybraną propozycję (okrąg) po prawej stronie zobaczymy słowa powiązane z tym tytułem i ich wagi.

## Zadanie

Wynik mógłby być lepszy. Pierwszą rzeczą, którą napewno warto zrobić pod kątem poprawienia wyniku jest usunięcie tokenów, które nie niosą ze sobą wiele znaczenia, a nie było ich na liście stopwords. W przypadku propozycji tytułu numer 1 będą to napewno tokeny takie jak 'f', 'x', 'g', 'b', '`', ... Warto też pewnie zmniejszyć liczbę słów, z których próbujemy zbudować tytuł. Zostawmy to ćwiczenie do samodzielnego rozwiązania, bo jest ono dobrym sposobem na utrwalenie materiału.


In [28]:
print(cleaned_articles[0][:500])

['mathemat', 'integr', 'continu', 'analog', 'sum', 'use', 'calcul', 'area', 'volum', 'gener', 'integr', 'process', 'comput', 'integr', 'one', 'two', 'fundament', 'oper', 'calculu', 'differenti', 'integr', 'start', 'method', 'solv', 'problem', 'mathemat', 'physic', 'find', 'area', 'curv', 'determin', 'displac', 'veloc', 'today', 'integr', 'use', 'wide', 'varieti', 'scientif', 'field', 'integr', 'enumer', 'call', 'definit', 'integr', 'interpret', 'sign', 'area', 'region', 'plane', 'bound', 'graph', 'given', 'function', 'two', 'point', 'real', 'line', 'convent', 'area', 'horizont', 'axi', 'plane', 'posit', 'area', 'neg', 'integr', 'also', 'refer', 'concept', 'antideriv', 'function', 'whose', 'deriv', 'given', 'function', 'case', 'also', 'call', 'indefinit', 'integr', 'fundament', 'theorem', 'calculu', 'relat', 'definit', 'integr', 'differenti', 'provid', 'method', 'comput', 'definit', 'integr', 'function', 'antideriv', 'known', 'differenti', 'integr', 'invers', 'oper', 'although', 'method

In [38]:
filtered_articles = [[token for token in article if re.match(r'\w{3,}', token)] for article in cleaned_articles]
print(filtered_articles[0][:5000])

['mathemat', 'integr', 'continu', 'analog', 'sum', 'use', 'calcul', 'area', 'volum', 'gener', 'integr', 'process', 'comput', 'integr', 'one', 'two', 'fundament', 'oper', 'calculu', 'differenti', 'integr', 'start', 'method', 'solv', 'problem', 'mathemat', 'physic', 'find', 'area', 'curv', 'determin', 'displac', 'veloc', 'today', 'integr', 'use', 'wide', 'varieti', 'scientif', 'field', 'integr', 'enumer', 'call', 'definit', 'integr', 'interpret', 'sign', 'area', 'region', 'plane', 'bound', 'graph', 'given', 'function', 'two', 'point', 'real', 'line', 'convent', 'area', 'horizont', 'axi', 'plane', 'posit', 'area', 'neg', 'integr', 'also', 'refer', 'concept', 'antideriv', 'function', 'whose', 'deriv', 'given', 'function', 'case', 'also', 'call', 'indefinit', 'integr', 'fundament', 'theorem', 'calculu', 'relat', 'definit', 'integr', 'differenti', 'provid', 'method', 'comput', 'definit', 'integr', 'function', 'antideriv', 'known', 'differenti', 'integr', 'invers', 'oper', 'although', 'method

In [39]:
dictionary_filtered = Dictionary(filtered_articles)
bow_corpus_filtered = [dictionary_filtered.doc2bow(article) for article in filtered_articles]

tfidf_model_filtered = TfidfModel(bow_corpus_filtered)
tfidf_corpus_filtered = tfidf_model_filtered[bow_corpus_filtered]

In [41]:
lda_model_filtered = LdaModel(
    corpus=tfidf_corpus_filtered,
    id2word=dictionary_filtered,
    num_topics=10,
    random_state=100,
    update_every=1,
    chunksize=100,
    passes=10,
    alpha="auto"
)

pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim.prepare(
    lda_model_filtered, 
    tfidf_corpus_filtered, 
    dictionary_filtered, 
    mds="mmds", 
    R=5  # liczba tokenów branych pod uwagę podczas analizy tytułu dla wybranego artykułu
)

vis

/home/mdabrowski-phd/.local/lib/python3.10/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.000982  0.003209       1        1  27.810877
0      0.000227 -0.001807       2        1  19.528236
7     -0.001030 -0.000495       3        1  17.612407
8     -0.000587  0.000217       4        1  11.811175
6      0.000428 -0.000431       5        1  10.042937
1      0.000306 -0.000188       6        1   8.311695
3     -0.000040  0.000078       7        1   3.051650
5     -0.000110 -0.000207       8        1   0.636740
9     -0.000120 -0.000197       9        1   0.597143
4     -0.000057 -0.000179      10        1   0.597139, topic_info=          Term      Freq     Total Category  logprob  loglift
372     integr  0.000000  0.000000  Default   5.0000   5.0000
299   function  0.000000  0.000000  Default   4.0000   4.0000
5452     kafka  0.000000  0.000000  Default   3.0000   3.0000
4952   tolstoy  0.000000  0.000000  Default   2.0000   2.0000
2084     munch  0.000000  0.000000  Default   1.0000   1.0000
...        ...       ...       ...      ...      ...      ...
372     integr  0.000101  0.096064  Topic10  -8.7971  -1.7386
299   function  0.000101  0.065243  Topic10  -8.7971  -1.3517
4952   tolstoy  0.000101  0.043995  Topic10  -8.7971  -0.9577
2084     munch  0.000101  0.029227  Topic10  -8.7971  -0.5487
184      deriv  0.000101  0.044855  Topic10  -8.7971  -0.9770

[85 rows x 6 columns], token_table=Empty DataFrame
Columns: [Topic, Freq, Term]
Index: [], R=5, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 8, 9, 7, 2, 4, 6, 10, 5])